### Todos
> This are the things which people can contribute in

* Create functions and optimize them
    * IDEA : They can create a recursive functions to unnest json data (Obviously for practice)



Author : @sid-146

In [1]:
import pandas as pd
import json
import numpy

#### Functions which will be used while cleaning process

In [2]:
# Removes any kind of special chars in the data

def sanitize_data(data):
    # Cleans only if data is string type
    if isinstance(data, str):
        clean_data = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in data)
        return clean_data
    else:
        return data


In [3]:
def apply_scorecard(row):
    match = row.match

    scorecard = mapping_dict.get(match, 'not found')

    return scorecard

#### Cleaning Batter Summary

In [4]:
with open('t20_json_files/t20_wc_batting_summary.json', 'r') as file:
    batter_records = json.load(file)

# Go through data before processing (data exploration)

In [5]:
'''
    Data Format for battingSummary
    list [
        dict : list [
            dict
        ]
    ]
    ########################################################
    records [
            {
                battingSummary (For that particular match): [
                    {
                        key : value ... n key value pair
                    }
                ]
            }
        ]

'''
# Task 'Create a recursive function to unnest data'

'\n    Data Format for battingSummary\n    list [\n        dict : list [\n            dict\n        ]\n    ]\n    ########################################################\n    records [\n            {\n                battingSummary (For that particular match): [\n                    {\n                        key : value ... n key value pair\n                    }\n                ]\n            }\n        ]\n\n'

In [6]:
batterSummary = pd.DataFrame()


# This loop shows the hierarchy data json data
# Very slow :Remember you can use in build functions which are optimized for performance:
for match in batter_records:
    for summary in match.keys():
        for batter in match[summary]:
            batter_info = {}
            for label in batter.keys():
                batter_info[label] = batter[label]

            batterSummary = batterSummary.append(
                batter_info, ignore_index=True)


# batterSummary


In [7]:
# Alternate
"""
    ShortCut
        all_record
        for record in data:
        df = pd.DataFrame(data[0]['battingSummary'])
        df.head()
        This will work as same
"""


"\n    ShortCut\n        all_record\n        for record in data:\n        df = pd.DataFrame(data[0]['battingSummary'])\n        df.head()\n        This will work as same\n"

In [8]:
# Adding out - notout columnSanitize Data
batterSummary.insert(len(batterSummary.columns), 'out_notout', batterSummary.dismissal.apply(lambda x: 'out' if len(x) > 0 else 'notout'))
batterSummary = batterSummary.applymap(sanitize_data)

#### Bowler Summary Cleaning

In [9]:
with open('t20_json_files\\t20_wc_bowling_summary.json') as file:
    bowler_records = json.load(file)

In [10]:
bowlerSummary = pd.DataFrame()

for match in bowler_records:
    for summary in match.keys():
        for batter in match[summary]:
            bowler_info = {}
            for label in batter.keys():
                bowler_info[label] = batter[label]

            bowlerSummary = bowlerSummary.append(
                bowler_info, ignore_index=True)

In [11]:
# Sanitizing bowlerSummary
bowlerSummary = bowlerSummary.applymap(sanitize_data)

#### Match Result Data

In [12]:
with open('t20_json_files\\t20_wc_match_results.json') as file:
    match_records = json.load(file)

In [13]:
matchResults = pd.DataFrame(match_records[0]['matchSummary'])

# Sanitize
matchResults = matchResults.applymap(sanitize_data)

In [14]:
matchResults.shape

(45, 7)

#### Each player information

In [15]:
with open('t20_json_files\\t20_wc_player_info.json') as file:
    player_records = json.load(file)


In [16]:
playerInfo = pd.DataFrame(player_records)

# Sanitize

playerInfo = playerInfo.applymap(sanitize_data)

In [17]:
playerInfo.shape

(219, 6)

#### Creating Mapping dict to provide Foreign key to each table 

Will help in joining different tables

In [18]:
mapping_dict = {}

'''
    mapping dictionary format
        team1 Vs team2 : scorecard  
        team2 Vs team1 : scorecard  
'''

for index, row in matchResults.iterrows():
    
    key1 = f'{row.team1} Vs {row.team2}' 
    key2 = f'{row.team2} Vs {row.team1}' 

    mapping_dict[key1] = row.scorecard
    mapping_dict[key2] = row.scorecard


In [19]:
# Mapping keys in each table
# 2 methods

batterSummary['scorecard'] = batterSummary['match'].map(mapping_dict)
bowlerSummary['scorecard'] = bowlerSummary.apply(apply_scorecard, axis=1)

In [20]:
bowlerSummary.head(10)

,0s,4s,6s,bowlerName,bowlingTeam,economy,maiden,match,noBalls,overs,runs,wickets,wides,scorecard
0,7,0,0,Maheesh Theekshana,Sri Lanka,5 75,0,Namibia Vs Sri Lanka,0,4,23,1,2,T20I 1823
1,6,3,1,Dushmantha Chameera,Sri Lanka,9 75,0,Namibia Vs Sri Lanka,0,4,39,1,2,T20I 1823
2,6,3,1,Pramod Madushan,Sri Lanka,9 25,0,Namibia Vs Sri Lanka,0,4,37,2,0,T20I 1823
3,7,3,1,Chamika Karunaratne,Sri Lanka,9 00,0,Namibia Vs Sri Lanka,0,4,36,1,1,T20I 1823
4,8,1,1,Wanindu Hasaranga de Silva,Sri Lanka,6 75,0,Namibia Vs Sri Lanka,0,4,27,1,0,T20I 1823
5,1,1,0,Gerhard Erasmus,Namibia,8 00,0,Namibia Vs Sri Lanka,0,1,8,0,0,T20I 1823
6,13,1,0,David Wiese,Namibia,4 00,0,Namibia Vs Sri Lanka,0,4,16,2,0,T20I 1823
7,10,1,0,Bernard Scholtz,Namibia,4 50,0,Namibia Vs Sri Lanka,0,4,18,2,0,T20I 1823
8,6,3,0,Ben Shikongo,Namibia,7 33,1,Namibia Vs Sri Lanka,0,3,22,2,0,T20I 1823
9,7,0,0,JJ Smit,Namibia,5 33,0,Namibia Vs Sri Lanka,0,3,16,1,1,T20I 1823


In [21]:
batterSummary.head(10)

,4s,6s,SR,balls,batsmanName,battingPos,dismissal,match,runs,teamInnings,out_notout,scorecard
0,0,0,50 00,6,Michael van Lingen,1.0,c Pramod Madushan b Chameera,Namibia Vs Sri Lanka,3,Namibia,out,T20I 1823
1,1,0,100 00,9,Divan la Cock,2.0,c Shanaka b Pramod Madushan,Namibia Vs Sri Lanka,9,Namibia,out,T20I 1823
2,1,2,166 66,12,Jan Nicol Loftie Eaton,3.0,c Mendis b Karunaratne,Namibia Vs Sri Lanka,20,Namibia,out,T20I 1823
3,2,0,108 33,24,Stephan Baard,4.0,c DM de Silva b Pramod Madushan,Namibia Vs Sri Lanka,26,Namibia,out,T20I 1823
4,0,0,83 33,24,Gerhard Erasmus c,5.0,c Gunathilaka b PWH de Silva,Namibia Vs Sri Lanka,20,Namibia,out,T20I 1823
5,4,0,157 14,28,Jan Frylinck,6.0,run out Gunathilaka Mendis,Namibia Vs Sri Lanka,44,Namibia,out,T20I 1823
6,0,0,0 00,1,David Wiese,7.0,c Mendis b Theekshana,Namibia Vs Sri Lanka,0,Namibia,out,T20I 1823
7,2,2,193 75,16,JJ Smit,8.0,,Namibia Vs Sri Lanka,31,Namibia,notout,T20I 1823
8,1,0,90 00,10,Pathum Nissanka,1.0,c Smit b Shikongo,Namibia Vs Sri Lanka,9,Sri Lanka,out,T20I 1823
9,0,0,100 00,6,Kusal Mendis,2.0,c Green b Wiese,Namibia Vs Sri Lanka,6,Sri Lanka,out,T20I 1823


In [22]:
matchResults.head(10)

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,Oct 16 2022,T20I 1823
1,Netherlands,U A E,Netherlands,3 wickets,Geelong,Oct 16 2022,T20I 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,Oct 17 2022,T20I 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,Oct 17 2022,T20I 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,Oct 18 2022,T20I 1830
5,Sri Lanka,U A E,Sri Lanka,79 runs,Geelong,Oct 18 2022,T20I 1832
6,Ireland,Scotland,Ireland,6 wickets,Hobart,Oct 19 2022,T20I 1833
7,West Indies,Zimbabwe,West Indies,31 runs,Hobart,Oct 19 2022,T20I 1834
8,Netherlands,Sri Lanka,Sri Lanka,16 runs,Geelong,Oct 20 2022,T20I 1835
9,Namibia,U A E,U A E,7 runs,Geelong,Oct 20 2022,T20I 1836


In [23]:
playerInfo.head(10)

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie Eaton,Namibia,Left hand Bat,Right arm Medium Legbreak,Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus c,Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder,
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder,David Wiese joined a marked outflow of South A...
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder,
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter,
9,Kusal Mendis,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter,Blessed with a compact technique an aggressiv...


#### Exporting the cleaned data frame in csv file

In [24]:
path = f'csv'

batterSummary.to_csv(f'{path}/batterSummary.csv', index=False)
bowlerSummary.to_csv(f'{path}/bowlerSummary.csv', index=False)
matchResults.to_csv(f'{path}/matchResults.csv', index=False)
playerInfo.to_csv(f'{path}/playerInfo.csv', index=False)